In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import os
import torchvision.transforms as transforms
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.transforms import Compose
from torchvision.transforms import Resize
import torch.optim as optim
from torch.utils.data import DataLoader , TensorDataset
from torch import Tensor
#!pip install einops
!pip install timm
from torchvision.models import resnet18


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
from numpy import load
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')

# images = load('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/x_train.npy')
# labels = load('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/y_train.npy')

######### KL colab
images = load('/content/drive/MyDrive/Colab_data/data_256/x_train.npy')
labels = load('/content/drive/MyDrive/Colab_data/data_256/y_train.npy')

# images = load('x_train.npy')
# labels = load('y_train.npy')



x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
x_train = x_train.reshape((-1,3,256,256))
x_test = x_test.reshape((-1,3,256,256))

Mounted at /content/drive


In [3]:
import timm
from timm import create_model
timm.list_models(pretrained = True)


['bat_resnext26ts.ch_in1k',
 'beit_base_patch16_224.in22k_ft_in22k',
 'beit_base_patch16_224.in22k_ft_in22k_in1k',
 'beit_base_patch16_384.in22k_ft_in22k_in1k',
 'beit_large_patch16_224.in22k_ft_in22k',
 'beit_large_patch16_224.in22k_ft_in22k_in1k',
 'beit_large_patch16_384.in22k_ft_in22k_in1k',
 'beit_large_patch16_512.in22k_ft_in22k_in1k',
 'beitv2_base_patch16_224.in1k_ft_in1k',
 'beitv2_base_patch16_224.in1k_ft_in22k',
 'beitv2_base_patch16_224.in1k_ft_in22k_in1k',
 'beitv2_large_patch16_224.in1k_ft_in1k',
 'beitv2_large_patch16_224.in1k_ft_in22k',
 'beitv2_large_patch16_224.in1k_ft_in22k_in1k',
 'botnet26t_256.c1_in1k',
 'caformer_b36.sail_in1k',
 'caformer_b36.sail_in1k_384',
 'caformer_b36.sail_in22k',
 'caformer_b36.sail_in22k_ft_in1k',
 'caformer_b36.sail_in22k_ft_in1k_384',
 'caformer_m36.sail_in1k',
 'caformer_m36.sail_in1k_384',
 'caformer_m36.sail_in22k',
 'caformer_m36.sail_in22k_ft_in1k',
 'caformer_m36.sail_in22k_ft_in1k_384',
 'caformer_s18.sail_in1k',
 'caformer_s18.s

In [4]:
print(np.shape(x_train))
torch.cuda.is_available()

(2880, 3, 256, 256)


True

In [5]:



image_size = 256

# Define transformations
train_transform = transforms.Compose([
    transforms.ToPILImage(),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomResizedCrop((256, 256), scale=(0.7, 1.0), ratio=(0.9, 1.1)),  # Random crop and resize
#     transforms.RandomRotation(degrees=(-10, 10), fill=(0,0,0)),  # Random rotation with black fill
#     transforms.ColorJitter(brightness=0.1, contrast=0.5, saturation=0.1),  # Random color adjustments
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])



# Apply transformations to custom datasets
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
x_train_processed = torch.stack([train_transform(image) for image in x_train_tensor])
x_test_processed = torch.stack([val_transform(image) for image in x_test_tensor])


# Create DataLoader objects
batch_size = 32

dataset_train = TensorDataset( x_train_processed, Tensor(y_train).long() )
dataset_test = TensorDataset( x_test_processed, Tensor(y_test).long())

train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)




In [10]:
torch.cuda.empty_cache()

In [11]:
# Initialize model, loss, and optimizer
num_epochs = 30

train_loss_rec = np.zeros(num_epochs)
train_acc_rec =np.zeros(num_epochs)
test_acc_rec = np.zeros(num_epochs)
# model  = MobileViT(
#     image_size = (256, 256),
#     dims = [96, 120, 144],
#     channels = [16, 32, 48, 48, 64, 64, 80, 80, 96, 96, 384],
#     num_classes = 2).to(device)
model = timm.create_model('mobilevitv2_200.cvnets_in1k', pretrained=True,num_classes = 2).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.RAdam(model.parameters(), lr=3e-4, weight_decay=2e-2)

best_val_loss = float('inf')
best_model_path = 'best_model.pth'
# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")
    train_loss_rec[epoch] = running_loss/len(train_loader)
    # Evaluation
    model.eval()
    train_correct = 0
    train_total = 0
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        val_loss = 0.0
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            val_loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
          # Save the best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), best_model_path)
            print("Best model saved with validation loss: {:.4f}".format(best_val_loss))

    print(f"train_Accuracy: {100 * train_correct / train_total}%")
    print(f"test_Accuracy: {100 * test_correct / test_total}%")
    train_acc_rec[epoch] =train_correct / train_total
    test_acc_rec[epoch] =test_correct / test_total

Epoch 1/30, Loss: 0.6779850264390309
Best model saved with validation loss: 14.6941
train_Accuracy: 76.14583333333333%
test_Accuracy: 69.90291262135922%
Epoch 2/30, Loss: 0.5057874701089329
Best model saved with validation loss: 8.5215
train_Accuracy: 89.30555555555556%
test_Accuracy: 84.18862690707351%
Epoch 3/30, Loss: 0.26438623285955853
Best model saved with validation loss: 7.1303
train_Accuracy: 93.68055555555556%
test_Accuracy: 86.26907073509015%
Epoch 4/30, Loss: 0.17879128990073997
train_Accuracy: 91.80555555555556%
test_Accuracy: 84.0499306518724%
Epoch 5/30, Loss: 0.12994085401296615
train_Accuracy: 93.33333333333333%
test_Accuracy: 86.40776699029126%
Epoch 6/30, Loss: 0.11085919849574566
train_Accuracy: 94.51388888888889%
test_Accuracy: 85.99167822468793%
Epoch 7/30, Loss: 0.09341904176399111
train_Accuracy: 87.36111111111111%
test_Accuracy: 80.02773925104022%
Epoch 8/30, Loss: 0.1120506739968227
Best model saved with validation loss: 5.5935
train_Accuracy: 96.9791666666666

In [13]:
# x_test = load('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/x_test.npy')
# y_test = load('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/y_test.npy')

x_test = load('/content/drive/MyDrive/Colab_data/data_256/x_test.npy')

y_test = load('/content/drive/MyDrive/Colab_data/data_256/y_test.npy')


x_test = x_test.reshape((-1,3,256,256))
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
x_test_processed = torch.stack([test_transform(image) for image in x_test_tensor])

dataset_test = TensorDataset( x_test_processed, Tensor(y_test).long())
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)

In [14]:
 # Evaluation
# Reload the best model for future use
best_model = timm.create_model('mobilevitv2_200.cvnets_in1k', pretrained=False,num_classes = 2).to(device)
#best_model.fc = nn.Linear(num_ftrs, 10)  # Make sure to match the architecture

best_model.load_state_dict(torch.load(best_model_path))
best_model.eval()





test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = best_model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_acc =test_correct / test_total

In [15]:
from numpy import save

# save('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/MVP_train_loss_rec.npy', train_loss_rec)
# save('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/MVP_train_acc_rec.npy', train_acc_rec)
# save('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/MVP_val_acc_rec.npy', test_acc_rec)
# save('/content/drive/My Drive/SYDE675_Leaf_Disease_Classification/KL_data/MVP_test_acc.npy', test_acc)

save('/content/drive/MyDrive/Colab_data/data_256/MVP_train_loss_rec.npy', train_loss_rec)
save('/content/drive/MyDrive/Colab_data/data_256/MVP_train_acc_rec.npy', train_acc_rec)
save('/content/drive/MyDrive/Colab_data/data_256/MVP_val_acc_rec.npy', test_acc_rec)
save('/content/drive/MyDrive/Colab_data/data_256/MVP_test_acc.npy', test_acc)
print(test_acc)

0.9234184239733629
